In [72]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [76]:
#Get the spam data collection 
df_spamCollection = pd.read_csv('SpamCollection', sep='\t', names=['response','message'])

In [77]:
#View the first five records
df_spamCollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [78]:
#View more information about the spam data using describe method 
df_spamCollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [79]:
#View response using group by and describe method 
df_spamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [80]:
# Verify length of the messages and also add it as a new column (feature)
df_spamCollection['length'] = df_spamCollection['message'].apply(len)

In [81]:
#View first 5 messages with length
df_spamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [82]:
#View response using group by and describe method 
df_spamCollection.groupby('response').describe()

length                                                         
           count        mean        std   min    25%    50%    75%    max
response                                                                 
ham       4825.0   71.482487  58.440652   2.0   33.0   52.0   93.0  910.0
spam       747.0  138.670683  28.873603  13.0  133.0  149.0  157.0  223.0

In [83]:
def message_text_process(mess):
    # Check characters to see if there are punctuations
    no_punctuation = [char for char in mess if char not in string.punctuation]
    #Now form the sentence
    no_punctuation = ''.join(no_punctuation)
    #Now eliminate any stopwords
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [85]:
#Verify that function is working
df_spamCollection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [86]:
#Start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [87]:
#Bag of words by applying the function and fit the data (message) into it
bag_of_words_transformer = CountVectorizer(analyzer=message_text_process).fit(df_spamCollection['message'])

In [89]:
#Print the length of bag of words stored in the vocabulary_attribute
print(len(bag_of_words_transformer.vocabulary_))      

11425


In [100]:
#Store bag of words for messages using transform method 
message_bagofwords = bag_of_words_transformer.transform(df_spamCollection['message'])

In [102]:
#Apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)

In [104]:
#Print shape of the tfidf
message_tfidf = tfidf_transformer.transform(message_bagofwords)
print(message_tfidf.shape)

(5572, 11425)


In [105]:
#Choose naive Bayes model to detect the spam and fit the tfidf data into it 
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf,df_spamCollection['response'])

In [108]:
#Check model for the predicted and expected value say for message #2 and message #5
message = df_spamCollection['message'][4]
bag_of_words_for_message = bag_of_words_transformer.transform([message])
tfidf = tfidf_transformer.transform(bag_of_words_for_message)

print('predicted', spam_detect_model.predict(tfidf)[0])
print('expected', df_spamCollection.response[4])



predicted ham
expected ham
